### Calculating Road Lengths

In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from requests import get
from urllib.parse import urlparse

In [2]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import matplotlib.pyplot as plt
import mapclassify as mc
import pysal as p
import palettable.matplotlib as palmpl
import seaborn as sns

from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage, set_link_color_palette

In [3]:
CamHS = gpd.read_file('Complete HS/CAMDEN_HS.shp')

In [4]:
CamHS

,objectid,highstreet,highstre_1,area_ha,gdb_geomat,layer,path,geometry
0,594,594,"Eversholt Street, Somers Town",6.303698,None,Eversholt,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((529676.878 182864.075, 529679.937 18..."
1,322,322,"Cricklewood Broadway, Cricklewood.",28.338859,None,Cricklewood,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((523415.282 186284.726, 523422.134 18..."
2,372,372,"Fortune Green Road, Fortune Green.",6.581487,None,Fortune,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((525198.994 185637.081, 525201.838 18..."
3,285,285,"Fortress Road, Tufnell Park.",28.325719,None,Fortress,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((529209.157 185757.682, 529206.506 18..."
4,303,303,"Belsize Lane, Belsize Park.",5.069615,None,Belsize,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((526892.645 185059.806, 526900.890 18..."
5,287,287,Camden Town HS,56.349258,None,CamdenT,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((528129.528 184489.100, 528138.350 18..."
6,97,97,Brecknock Road & York Way.,15.327961,None,Brecknock,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((529626.963 185290.470, 529619.385 18..."
7,468,468,"Mansfield Road, Gospel Oak",3.508042,None,Mansfield,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((528267.619 185602.755, 527975.356 18..."
8,304,304,"Loudoun Road, South Hampstead.",8.078871,None,Loudoun,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((526350.252 184351.109, 526360.006 18..."
9,328,328,"Queen?s Crescent, Gospel Oak.",10.314779,None,Queens,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((528422.038 185153.550, 528417.412 18..."


In [5]:
print(CamHS.crs)

epsg:27700


In [6]:
CamHS['highstreet_name'] = CamHS['layer']

In [7]:
CamHS['road_length'] = 0

In [8]:
CamHS['road_length'] = CamHS['road_length'].astype('float')

In [9]:
CamHS

,objectid,highstreet,highstre_1,area_ha,gdb_geomat,layer,path,geometry,highstreet_name,road_length
0,594,594,"Eversholt Street, Somers Town",6.303698,None,Eversholt,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((529676.878 182864.075, 529679.937 18...",Eversholt,0.0
1,322,322,"Cricklewood Broadway, Cricklewood.",28.338859,None,Cricklewood,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((523415.282 186284.726, 523422.134 18...",Cricklewood,0.0
2,372,372,"Fortune Green Road, Fortune Green.",6.581487,None,Fortune,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((525198.994 185637.081, 525201.838 18...",Fortune,0.0
3,285,285,"Fortress Road, Tufnell Park.",28.325719,None,Fortress,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((529209.157 185757.682, 529206.506 18...",Fortress,0.0
4,303,303,"Belsize Lane, Belsize Park.",5.069615,None,Belsize,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((526892.645 185059.806, 526900.890 18...",Belsize,0.0
5,287,287,Camden Town HS,56.349258,None,CamdenT,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((528129.528 184489.100, 528138.350 18...",CamdenT,0.0
6,97,97,Brecknock Road & York Way.,15.327961,None,Brecknock,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((529626.963 185290.470, 529619.385 18...",Brecknock,0.0
7,468,468,"Mansfield Road, Gospel Oak",3.508042,None,Mansfield,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((528267.619 185602.755, 527975.356 18...",Mansfield,0.0
8,304,304,"Loudoun Road, South Hampstead.",8.078871,None,Loudoun,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((526350.252 184351.109, 526360.006 18...",Loudoun,0.0
9,328,328,"Queen?s Crescent, Gospel Oak.",10.314779,None,Queens,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((528422.038 185153.550, 528417.412 18...",Queens,0.0


## Belsize

In [10]:
belsizeroad = gpd.read_file('High Street Roads/belsizeroad.shp')

In [11]:
belsizeroad = belsizeroad.to_crs('EPSG:27700')

In [12]:
belsizeroad = belsizeroad['geometry'].length.values

In [13]:
belsizeroad

array([357.54817745])

In [14]:
CamHS.loc[4, 'road_length'] = belsizeroad

In [23]:
CamHS

,objectid,highstreet,highstre_1,area_ha,gdb_geomat,layer,path,geometry,highstreet_name,road_length
0,594,594,"Eversholt Street, Somers Town",6.303698,None,Eversholt,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((529676.878 182864.075, 529679.937 18...",Eversholt,0.000000
1,322,322,"Cricklewood Broadway, Cricklewood.",28.338859,None,Cricklewood,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((523415.282 186284.726, 523422.134 18...",Cricklewood,0.000000
2,372,372,"Fortune Green Road, Fortune Green.",6.581487,None,Fortune,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((525198.994 185637.081, 525201.838 18...",Fortune,0.000000
3,285,285,"Fortress Road, Tufnell Park.",28.325719,None,Fortress,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((529209.157 185757.682, 529206.506 18...",Fortress,0.000000
4,303,303,"Belsize Lane, Belsize Park.",5.069615,None,Belsize,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((526892.645 185059.806, 526900.890 18...",Belsize,357.548177
5,287,287,Camden Town HS,56.349258,None,CamdenT,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((528129.528 184489.100, 528138.350 18...",CamdenT,0.000000
6,97,97,Brecknock Road & York Way.,15.327961,None,Brecknock,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((529626.963 185290.470, 529619.385 18...",Brecknock,1508.451692
7,468,468,"Mansfield Road, Gospel Oak",3.508042,None,Mansfield,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((528267.619 185602.755, 527975.356 18...",Mansfield,0.000000
8,304,304,"Loudoun Road, South Hampstead.",8.078871,None,Loudoun,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((526350.252 184351.109, 526360.006 18...",Loudoun,0.000000
9,328,328,"Queen?s Crescent, Gospel Oak.",10.314779,None,Queens,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((528422.038 185153.550, 528417.412 18...",Queens,0.000000


## Brecknock

In [16]:
brecknockroad = gpd.read_file('High Street Roads/brecknock.shp')

In [17]:
brecknockroad = brecknockroad.to_crs('EPSG:27700')

In [18]:
brecknockroad = brecknockroad['geometry'].length.values

In [21]:
brecknockroad

1508.4516917410836

In [20]:
brecknockroad = brecknockroad.sum()

In [22]:
CamHS.loc[6, 'road_length'] = brecknockroad

## Camden High Street

In [24]:
camdenHSroad = gpd.read_file('High Street Roads/camdenHSroad.shp')

In [25]:
camdenHSroad = camdenHSroad.to_crs('EPSG:27700')

In [27]:
camdenHSroad = camdenHSroad['geometry'].length.values

In [28]:
camdenHSroad

array([257.16399991, 227.99007574,  19.36996973,  47.8613845 ,
        80.53072523,  39.05883298,   9.21351331, 190.62375425,
        17.97462012,  64.36666899,  40.78499858,  12.86937339,
        28.18514986,  44.16556279, 144.60213961,  14.50403468,
        29.58148229,  58.59431591,  26.08648081,  55.63211503,
        33.37897148,  52.31246548,  65.72868304,  22.83231469,
       131.46808525,  41.88631855,  32.85985097,  47.86117433,
        14.16107442,   7.72792472,   7.56928412])

In [29]:
camdenHSroad = camdenHSroad.sum()

In [30]:
CamHS.loc[5, 'road_length'] = camdenHSroad

### Cricklewood

In [33]:
cricklewood = gpd.read_file('High Street Roads/cicklewood.shp')

In [34]:
cricklewood = cricklewood.to_crs('EPSG:27700')

In [35]:
cricklewood = cricklewood['geometry'].length.values

In [36]:
cricklewood

array([451.97596179])

In [37]:
CamHS.loc[1, 'road_length'] = cricklewood

### Eversholt

In [38]:
eversholt = gpd.read_file('High Street Roads/eversholt(idk bout this one).shp')

In [39]:
eversholt = eversholt.to_crs('EPSG:27700')

In [40]:
eversholt = eversholt['geometry'].length.values

In [41]:
eversholt

array([169.92377829,   6.13983082,  20.72828952, 121.45470445,
       670.51564769,   9.4696615 ])

In [42]:
eversholt = eversholt.sum()

In [114]:
CamHS.loc[0, 'road_length'] = eversholt

### Fortress

In [44]:
fortress = gpd.read_file('High Street Roads/fortress.shp')

In [45]:
fortress = fortress.to_crs('EPSG:27700')

In [46]:
fortress = fortress['geometry'].length.values

In [47]:
fortress

array([264.64031446,  90.52248933,   3.23223084,  35.47672642,
        55.95049128,  35.0676301 , 267.75353571,   9.21685584,
        24.62598211, 191.6547591 ,   6.72566041,  30.01469004,
       552.99443684,  12.13951363,  11.10795911,  22.97252902,
        14.03855394])

In [48]:
fortress = fortress.sum()

In [49]:
CamHS.loc[3, 'road_length'] = fortress

### Fortune Green

In [50]:
fortune = gpd.read_file('High Street Roads/fortunegreen.shp')

In [51]:
fortune = fortune.to_crs('EPSG:27700')

In [52]:
fortune = fortune['geometry'].length.values

In [53]:
fortune

array([ 18.76495006, 617.4131902 ,  24.95538553, 110.39678944])

In [54]:
fortune = fortune.sum()

In [55]:
CamHS.loc[2, 'road_length'] = fortune

### Hampstead

In [56]:
hampstead = gpd.read_file('High Street Roads/hampsteadHS.shp')

In [57]:
hampstead = hampstead.to_crs('EPSG:27700')

In [58]:
hampstead = hampstead['geometry'].length.values

In [59]:
hampstead

array([ 61.76872889, 275.06385701,  96.95603774, 343.60046581,
        68.93098447, 429.58931453, 372.60308163,  66.62789411,
        24.54894444, 128.68399662])

In [60]:
hampstead = hampstead.sum()

In [115]:
CamHS.loc[12, 'road_length'] = hampstead

### Haverstock Hill

In [62]:
haverstock = gpd.read_file('High Street Roads/haverstockhill.shp')

In [63]:
haverstock = haverstock.to_crs('EPSG:27700')

In [64]:
haverstock = haverstock['geometry'].length.values

In [65]:
haverstock

array([36.0793006 , 71.97791148, 29.05778905, 81.42768971, 14.51199218,
       28.68609476, 78.99396006, 30.69420484, 85.00185501,  7.13602567,
       20.64194813, 28.09224663])

In [66]:
haverstock = haverstock.sum()

In [67]:
CamHS.loc[11, 'road_length'] = haverstock

### Kentish Town

In [68]:
kentish = gpd.read_file('High Street Roads/kentishroad.shp')

In [69]:
kentish = kentish.to_crs('EPSG:27700')

In [70]:
kentish = kentish['geometry'].length.values

In [71]:
kentish

array([  8.75326003,   4.96085883,  10.3110021 ,  68.28341081,
        32.22026984,   7.93664443,  26.63819123, 158.31290656,
         3.88669443,  24.63718263,   3.54693584,  14.77813029,
        15.50433501,  88.3139694 ,  14.44690499,  11.60160154,
        34.49043713,  43.65671791,   4.52917193,   6.94583404,
        12.33594872])

In [72]:
kentish = kentish.sum()

In [73]:
CamHS.loc[14, 'road_length'] = kentish

### Kilburn

In [74]:
kilburn = gpd.read_file('High Street Roads/kilburn (some missing).shp')

In [75]:
kilburn = kilburn.to_crs('EPSG:27700')

In [76]:
kilburn = kilburn['geometry'].length.values

In [77]:
kilburn

array([157.10135755, 456.06144705, 285.09667483, 692.43521474,
       168.76822634,  57.93002535,  20.84054416,  31.67638188,
        16.44342888,  11.81723641,  52.75368784,  57.96987228])

In [78]:
kilburn = kilburn.sum()

In [79]:
CamHS.loc[13, 'road_length'] = kilburn

### Loudoun Road

In [80]:
loudoun = gpd.read_file('High Street Roads/loudounroad.shp')

In [81]:
loudoun = loudoun.to_crs('EPSG:27700')

In [82]:
loudoun = loudoun['geometry'].length.values

In [83]:
loudoun

array([  1.79368361,  25.35887178,  50.790073  , 404.47382133,
       147.44342577,  17.27174378,  35.46221669])

In [84]:
loudoun = loudoun.sum()

In [85]:
CamHS.loc[8, 'road_length'] = loudoun

### Mansfield

In [86]:
mansfield = gpd.read_file('High Street Roads/mansfield.shp')

In [87]:
mansfield = mansfield.to_crs('EPSG:27700')

In [88]:
mansfield = mansfield['geometry'].length.values

In [89]:
mansfield

array([ 61.76872889,  96.95603774, 429.58931453])

In [90]:
mansfield = mansfield.sum()

In [91]:
CamHS.loc[7, 'road_length'] = mansfield

### Mill Lane

In [93]:
mill = gpd.read_file('High Street Roads/mill lane.shp')

In [94]:
mill = mill.to_crs('EPSG:27700')

In [95]:
mill = mill['geometry'].length.values

In [96]:
mill

array([195.80056772, 374.83709743,  83.81059785])

In [97]:
mill = mill.sum()

In [98]:
CamHS.loc[10, 'road_length'] = mill

### Queen's Crescent

In [100]:
queens = gpd.read_file('High Street Roads/queenscrescent.shp')

In [101]:
queens = queens.to_crs('EPSG:27700')

In [102]:
queens = queens['geometry'].length.values

In [103]:
queens

array([ 60.96717852,  85.25019558,  53.40335005,  53.47098911,
        22.53726356, 391.63775958, 515.37516138, 119.00369568,
        41.81977168,  68.55121762,  47.92995406,  10.59373326,
         1.95201732,   2.91854029])

In [104]:
queens = queens.sum()

In [105]:
CamHS.loc[9, 'road_length'] = queens

### Regents Park

In [107]:
regents = gpd.read_file('High Street Roads/regentsroad.shp')

In [108]:
regents = regents.to_crs('EPSG:27700')

In [109]:
regents = regents['geometry'].length.values

In [110]:
regents

array([324.90186288,  59.12115303,  10.10808444])

In [111]:
regents = regents.sum()

In [112]:
CamHS.loc[16, 'road_length'] = regents

### Southend

In [117]:
southend = gpd.read_file('High Street Roads/southend.shp')

In [118]:
southend = southend.to_crs('EPSG:27700')

In [119]:
southend = southend['geometry'].length.values

In [120]:
southend

array([  5.34008207, 180.14501703,  64.98154358,  53.07294952,
        26.10044063,  27.57540417, 219.03059567])

In [121]:
southend = southend.sum()

In [122]:
CamHS.loc[15, 'road_length'] = southend

### West End

In [124]:
westend = gpd.read_file('High Street Roads/westend.shp')

In [125]:
westend = westend.to_crs('EPSG:27700')

In [126]:
westend = westend['geometry'].length.values

In [127]:
westend

array([ 49.4008505 ,  37.31366234,  42.8640761 ,  16.65201091,
        35.61084408,  24.74927708,  44.08864598,  19.10872665,
        48.45125995,  26.45589503,  46.50842384, 456.28552704,
       142.84788088,  82.69082193,  15.66402685,  68.26998392,
        48.26586759,  73.48843357, 145.84934756,  45.13612474,
        80.8631711 ,  12.6632344 ,   5.76974848,   9.43614187,
        21.91666255, 147.87433804, 173.59717481,  80.32587831,
       193.32104677,  38.42426263,  70.95288553, 136.20275545,
         8.77517799,  18.71195928])

In [128]:
westend = westend.sum()

In [129]:
CamHS.loc[18, 'road_length'] = westend

### Willesden

In [130]:
willesden = gpd.read_file('High Street Roads/willesden.shp')

In [131]:
willesden = willesden.to_crs('EPSG:27700')

In [132]:
willesden = willesden['geometry'].length.values

In [133]:
willesden

array([541.328572  , 182.70065931])

In [134]:
willesden = willesden.sum()

In [135]:
CamHS.loc[17, 'road_length'] = willesden

In [136]:
CamHS

,objectid,highstreet,highstre_1,area_ha,gdb_geomat,layer,path,geometry,highstreet_name,road_length
0,594,594,"Eversholt Street, Somers Town",6.303698,None,Eversholt,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((529676.878 182864.075, 529679.937 18...",Eversholt,998.231912
1,322,322,"Cricklewood Broadway, Cricklewood.",28.338859,None,Cricklewood,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((523415.282 186284.726, 523422.134 18...",Cricklewood,451.975962
2,372,372,"Fortune Green Road, Fortune Green.",6.581487,None,Fortune,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((525198.994 185637.081, 525201.838 18...",Fortune,771.530315
3,285,285,"Fortress Road, Tufnell Park.",28.325719,None,Fortress,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((529209.157 185757.682, 529206.506 18...",Fortress,1628.134358
4,303,303,"Belsize Lane, Belsize Park.",5.069615,None,Belsize,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((526892.645 185059.806, 526900.890 18...",Belsize,357.548177
5,287,287,Camden Town HS,56.349258,None,CamdenT,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((528129.528 184489.100, 528138.350 18...",CamdenT,1866.945345
6,97,97,Brecknock Road & York Way.,15.327961,None,Brecknock,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((529626.963 185290.470, 529619.385 18...",Brecknock,1508.451692
7,468,468,"Mansfield Road, Gospel Oak",3.508042,None,Mansfield,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((528267.619 185602.755, 527975.356 18...",Mansfield,588.314081
8,304,304,"Loudoun Road, South Hampstead.",8.078871,None,Loudoun,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((526350.252 184351.109, 526360.006 18...",Loudoun,682.593836
9,328,328,"Queen?s Crescent, Gospel Oak.",10.314779,None,Queens,C:/Users/CHRISTINA/OneDrive - University Colle...,"POLYGON ((528422.038 185153.550, 528417.412 18...",Queens,1475.410828


In [138]:
CamHS.to_excel("outputs_csv/road lengths.xlsx")